In [1]:
import math
import web3
import json
from Pool import Pool
from Object import Object
import Archive
import random
import hashlib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
0.11204969029*4000

448.19876116

In [3]:
network_url = 'https://ropsten.infura.io/v3/55a512fc01d84f59aa61929ffc8dff87'
w3 = web3.Web3(web3.HTTPProvider(network_url))
sender = "0x18648B486Bd6B771DB957590E988A2464F22BfCd"
# pk generated from truffle, unsafe to use
private_key="57b21d9e3d36c19c548091acd195c91ebe3ed27dec0fe4903d008f9af9e72f39"
balance = w3.fromWei(w3.eth.get_balance(sender),"ether")
tx_count = w3.eth.getTransactionCount(sender)
contract_address = "0x0243c7aa552730e8c6f7ed25a480a7c0c88a70f0"
print("Transaction Count {}".format(tx_count))
print("ETH Balance: {}".format(balance))
print("Sender Account: {}".format(sender))
compiled_contract_path = '../sol/build/contracts/FixityStorage.json'
# check contract address if this cell fails
deployed_contract_address = w3.toChecksumAddress(contract_address)
print("Contract Deployed at: {}".format(deployed_contract_address))

with open(compiled_contract_path) as file:
    contract_json = json.load(file)  # load contract info as JSON
    contract_abi = contract_json['abi']  # fetch contract's abi - necessary to call its functions

# Fetch deployed contract reference
contract = w3.eth.contract(address=deployed_contract_address, abi=contract_abi)
print("Contract Functions: {}".format(contract.all_functions()))

Transaction Count 64
ETH Balance: 0.02271006499
Sender Account: 0x18648B486Bd6B771DB957590E988A2464F22BfCd
Contract Deployed at: 0x0243c7aa552730E8C6F7ED25A480a7C0c88a70f0
Contract Functions: [<Function getPoolHash(uint32)>, <Function setPoolHash(uint32,bytes32)>]


In [4]:
def optimal_size(p):
    # https://www.sciencedirect.com/science/article/pii/S1201971220306925
    return math.ceil(1.24 * math.pow(p,-0.466))

In [5]:
encoding = "utf-8"
N = 100
percent = 10
prevalence = (percent * N) /100.0
p = percent/100.0

objects = [Object(i,random.uniform(0,p)) for i in range(0,N)]
assert objects[2].hash == hashlib.sha256("2".encode(encoding)).hexdigest()

k = optimal_size(prevalence/N)

pools=[]
for i in range((len(objects) + k - 1) // k ):
    # assign pool id to each object in the pool
    for obj in objects[i * k:(i + 1) * k]:
        obj.pool_id=i
    pools.append(Pool(objects[i * k:(i + 1) * k],i))

assert pools[0].id == 0
assert len(pools[0].objects) == len(pools[1].objects)
assert pools[len(pools)-1].hash

The following code cell is the basis for the calculation of txdatazero and txdatazero in Section 5.6 Cost of interacting in my thesis

In [7]:
input_data = "0x178d292900000000000000000000000000000000000000000000000000000000000000001760d27083f6e2d1c46a65938c03a0c52dccf55cb4eb68a720e6efe3a8851f78"
zeros =  input_data.count("0")
non_zeros =  len(input_data)-zeros
gas_setPoolHash = zeros*4+non_zeros*16
gas_sset = 20000
gas_transaction = 21000
gas = gas_setPoolHash+gas_sset+gas_transaction
print("Estimated gas cost the transaction={}".format(gas))

Estimated gas cost the transaction=42368


In [8]:
# gas price taken from anyblocks
## https://dashboards.anyblock.tools/ethereum/ethereum/ropsten/gas-metrics/?currency=usd&chunk_size=hour&interval=latest_week&speed=slow&tx_type=NATIVE&tx_currency=usd&tx_chunk_size=hour
gasPrice = w3.toWei("86.11","gwei")
print("Current Gas Price in gwei={} and wei={}".format("86.11",gasPrice))
gasLimit = int(gas + gas/100*20)
print("Maximum amount of gas I am willing to pay for the tx={}".format(gasLimit))


Current Gas Price in gwei=86.11 and wei=86110000000
Maximum amount of gas I am willing to pay for the tx=50841


In [11]:
gas

42368

In [17]:
w3.fromWei(w3.toWei("42","gwei"),"ether")

Decimal('4.2E-8')

In [10]:
ethPrice = 2838
gasPriceEth=w3.fromWei(w3.toWei("42","gwei"),"ether")
format(gas*gasPriceEth*ethPrice,"f")

'5.050096128'

In [1]:
42368 * 0.00000005 * 4000

8.4736

In [159]:
# persist each pool on the blockchain, for each pool perform a transaction
from tqdm.notebook import tqdm, trange
write_tx_count = 0
read_tx_count = 0
transactions = []
nonce = w3.eth.getTransactionCount(sender)
for pool in tqdm(pools):
    tx = contract.functions.setPoolHash(pool.id,pool.hash).buildTransaction(
        {
            "nonce":nonce,
            "gas": int(gas + gas/100*20),
            "gasPrice": gasPrice
        }
    )
    signed_tx = w3.eth.account.sign_transaction(tx,private_key=private_key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
    transactions.append(tx_hash)
    nonce = nonce + 1
    write_tx_count = write_tx_count + 1 


  0%|          | 0/25 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [160]:
import time
w3.eth.wait_for_transaction_receipt(transactions[len(transactions)-1].hex(),timeout=120)
# wait for the transaction to settle, otherwise the assert will throw because the pools are not updated on chain
time.sleep(5)
pool_id = pools[0].id
poolHashBytes = contract.functions.getPoolHash(pool_id).call()
assert poolHashBytes.hex() == pools[pool_id].hash

In [161]:
import importlib
importlib.reload(Archive)
archive = Archive.Archive(objects)

assert archive.objects[k*2].pool_id == 2 
assert Pool(archive.get_objects_by_pool_id(2)).hash == pools[2].hash
assert objects[2].hash==archive.retrieveObj(2).hash
# write transactions have to be exactly the number of pools at this stage
assert write_tx_count == len(pools)
sample = archive.retrieveObj(0)
assert sample.pool_id==0
pool_of_sample = Pool(archive.get_objects_by_pool_id(sample.pool_id))
assert pool_of_sample.hash == pools[0].hash
pool_in_blockchain = contract.functions.getPoolHash(sample.pool_id).call()
assert pool_of_sample.hash == pool_in_blockchain.hex()

In [162]:
archive.corrupt(p)

In [163]:
# repair every object in the archive
already_cleaned_pool_ids = set()
corrupted_objects_count = 0
for obj in tqdm(archive.objects):
    pool_of_sample = Pool(archive.get_objects_by_pool_id(obj.pool_id))
    pool_in_blockchain = contract.functions.getPoolHash(obj.pool_id).call()

    # is the local pool hash the same as the one in the blockchain? and make sure to not double repair a pool
    if(pool_of_sample.hash != pool_in_blockchain.hex() and obj.pool_id not in already_cleaned_pool_ids):
        tx = contract.functions.setPoolHash(pool.id,pool.hash).buildTransaction(
                {
                    "nonce":nonce,
                    "gas": int(gas + gas/100*20),
                    "gasPrice": gasPrice
                }
            )
        signed_tx = w3.eth.account.sign_transaction(tx,private_key=private_key)
        tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
        already_cleaned_pool_ids.add(obj.pool_id)
        write_tx_count = write_tx_count+1
        nonce = nonce + 1

print("Number of Distinct Cleaned Pools: {}".format(len(already_cleaned_pool_ids)))
print("Write transactions after cleaning: {}".format(write_tx_count))

  0%|          | 0/100 [00:00<?, ?it/s]

Number of Distinct Cleaned Pools: 8
Write transactions after cleaning: 33


In [166]:
fin_balance = w3.fromWei(w3.eth.get_balance(sender),"ether")
fin_tx_count = w3.eth.getTransactionCount(sender)
print("Optimal poolsize {} with prevalence {} in N={}".format(k,prevalence,N))
print("{} objects distributed in {} pools with size={} last pool with size={}".format(N,len(pools),k,len(pools[len(pools)-1].objects)))
print("Write Transaction Count: {} ".format(fin_tx_count - tx_count))
print("Total Cost=ETH {} for {} transactions ".format((balance - fin_balance),(fin_tx_count - tx_count)))
print("Theoretical amount of write transactions: {} with N={} + prevalence={}".format((N+prevalence),N,prevalence))
print("Number of Distinct Cleaned Pools: {}".format(len(already_cleaned_pool_ids)))

Optimal poolsize 4 with prevalence 10.0 in N=100
100 objects distributed in 25 pools with size=4 last pool with size=4
Write Transaction Count: 33 
Total Cost=ETH 0.11204969029 for 33 transactions 
Theoretical amount of write transactions: 110.0 with N=100 + prevalence=10.0
Number of Distinct Cleaned Pools: 8
